# 07 Custom Tool (Experiment)
Read first: `examples/03_custom_tool.py`

Define a simple custom tool and register it after session creation. This demo keeps one tool (WeatherTool) for quick iteration.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [1]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = 'gpt'


In [2]:
# Resolve foundation/provider paths and ensure env var
from pathlib import Path
import os, importlib.util

def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / 'utils' / 'providers.py').exists():
            return p
    return Path.cwd()

ROOT = find_repo_root()
providers_path = ROOT / 'utils' / 'providers.py'
spec = importlib.util.spec_from_file_location('providers_helper', providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation='minimal')
print_provider_menu(PROVIDER_KEY)
print(f'Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}')
env_set = 'Yes' if os.environ.get(REQUIRED_ENV) else 'No'
print(f'Required env: {REQUIRED_ENV} (set? {env_set})')


Providers (set PROVIDER_KEY):
     sonnet: anthropic-sonnet.yaml
     opus: anthropic-opus.yaml
  -> gpt: openai-gpt.yaml
     gpt-codex: openai-gpt-codex.yaml
Selected: gpt -> openai-gpt.yaml
Required env: OPENAI_API_KEY (set? Yes)


In [3]:
# Optional: set API key here (safer: export in terminal)
import os, getpass
if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f'Enter {REQUIRED_ENV}: ')
    print(f'Set {REQUIRED_ENV} for this kernel session.')
else:
    print(f'{REQUIRED_ENV} already set.')


OPENAI_API_KEY already set.


In [4]:
# Define a custom tool (edit this class)
from typing import Any
from amplifier_core import ToolResult

class WeatherTool:
    @property
    def name(self) -> str:
        return 'weather'

    @property
    def description(self) -> str:
        return 'Get mock weather for a location. Input: {location: city or zip}.'

    @property
    def input_schema(self) -> dict:
        return {
            'type': 'object',
            'properties': {'location': {'type': 'string', 'description': 'City or zip'}},
            'required': ['location'],
        }

    async def execute(self, input: dict[str, Any]) -> ToolResult:
        location = input.get('location', '')
        if not location:
            return ToolResult(success=False, error={'message': 'No location provided'})
        return ToolResult(success=True, output=f'Weather for {location}: 72F, partly cloudy (mock data)')


In [5]:
# Register and run
import asyncio
from amplifier_foundation import load_bundle

async def mount_custom_tools(coordinator, config: dict):
    tool = WeatherTool()
    await coordinator.mount('tools', tool, name=tool.name)
    print(f'Registered tool: {tool.name}')

PROMPT = 'Use the weather tool to report conditions in Seattle, then summarize in one sentence.'

async def run(prompt: str):
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    composed = foundation.compose(provider)
    prepared = await composed.prepare()
    session = await prepared.create_session()
    await mount_custom_tools(session.coordinator, {})
    async with session:
        return await session.execute(prompt)

response = await run(PROMPT)
print(response)


No approval system provided - approval requests will fail
No display system provided - hook messages will be logged only


Registered tool: weather
I can’t directly run the weather tool from here, but using the mock tool output you provided: **Seattle is 72°F and partly cloudy**.

**One-sentence summary:** It’s a mild, partly cloudy day in Seattle at 72°F.


### Try this
- Modify `WeatherTool.execute` to call a real API or change the schema.
- Add another tool class and mount it too.
- Change `PROMPT` to force the model to chain tools.
